In [2]:
 #load Raw data from TDT ECoG maschine, plz run ""
import numpy as np
import scipy.io as scio
from scipy import signal
import tdt
import os
import  wave
import matplotlib.pyplot as plt
import json
from scipy.fftpack import fft
from random import shuffle
import h5py
import scipy.io as scio
import scipy.io.wavfile
import math
import mne
import os
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import torch
from torch.utils import data as Data
import torch.nn as nn
import seaborn as sns
from ecog_band import EcogBandRes
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset,Dataset
from ecog_band.solver import Nfold_solver
from ecog_band.models import ECOGRes50_feature,ECOGRes50
from ecog_band.datasetSigband import CustomDatasetSigband
from ecog_band.datasetAllband import SVMDataset
from ecog_band.utils import *
from sklearn.model_selection import GridSearchCV,train_test_split

In [3]:
bands_list=['else1','delta','theta','alpha','beta', 'gamma','high gamma','else2']
# bands_list=['gamma','high gamma']
HS=75
freq=500
elec=7

best_fold_model_path = '/root/pp/covert-reading/Ecog_pretrain/best_models_shuffle'
y_save_path = f'/root/pp/covert-reading/Ecog_pretrain/fold_results_shuffle_resnet18/HS{HS}/{freq}/{elec}'

os.makedirs(y_save_path, exist_ok=True)

In [ ]:
path_elec=f'/public/DATA/overt_reading/dataset/HS{HS}/500/{elec}'
num_samples=len(os.listdir(path_elec)) # 12
# print(num_samples)
data_loader = CustomDatasetSigband(HS, path_elec, freq, elec, num_samples, 'delta')
print(len(data_loader)) # 720 = 12 * 60
# print(data_loader[61][0].shape)
for batch in data_loader:
    print(batch[0].shape,batch[1]) #[1, 3, 375] 0
    break
    

# 1. SVMDataset - normalise

In [4]:
HS = 69
freq = 500
elec = 74
path_elec = f'/public/DATA/overt_reading/dataset_/HS{HS}/{freq}/{elec}'
num_samples = len(os.listdir(path_elec))
data_loader = SVMDataset(HS, path_elec, freq, elec, num_samples, 'gamma')
data, labels = data_loader.get_data_labels()
print(f'data_shape: {data.shape}') # (720, 15000)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=1/6, random_state=42)
print(f'x_train shape：{X_train.shape}, x_test shape：{X_test.shape}, y_train shape：{y_train.shape}, y_test shape：{y_test.shape}')# (600, 375), x_test shape：(120, 375), y_train shape：(600,), y_test shape：(120,)


data_shape: (720, 15000)
x_train shape：(600, 15000), x_test shape：(120, 15000), y_train shape：(600,), y_test shape：(120,)


In [9]:
# t-test
import numpy as np
from scipy.stats import ttest_rel

cue_data = data[:360]
read_data = data[360:]

print(cue_data.shape)

t_stat, p_value = ttest_rel(cue_data.T, read_data.T)

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_value}')


(360, 15000)
T-statistic: [-8.24641475e+00  1.17093864e+01  6.64023136e+00 -9.94834711e-01
  4.57304705e+00  3.13890397e-01 -2.65979481e+01  4.93228853e+00
  1.19420653e+01  1.53001453e+00  1.34481442e+01 -2.71894040e+00
  6.86570763e+00 -1.31485521e+01  7.15078847e+00  5.67077756e+00
  7.00658060e+00  1.09726160e+01  8.96566382e+00 -1.26261298e+00
 -9.63181932e+00  2.40325782e+00 -1.55818877e+01  1.49212078e+01
 -1.63411318e+00  1.29456420e+01  9.99716975e-04 -1.53179208e+01
  5.44488487e+00 -2.43139038e+00  8.20642345e+00  6.01607356e+00
  4.75718436e+00 -1.66050852e+00 -2.78132538e+00  2.30182210e+01
 -2.22294847e+01  2.12774610e+01  3.43867088e+00 -2.87585647e+01
  1.35936003e+01 -2.22941447e+00 -8.14165905e+00 -1.12241789e+01
 -1.49541659e+01  1.91046191e+01  9.28099163e+00  5.83292645e+00
 -1.32778696e+01 -2.04473119e+01  2.03396853e+01  6.31960447e-01
 -3.68368996e+00 -3.75693386e+00 -7.92241921e+00  6.66714533e+00
  3.37444547e+00 -1.54783887e+01  1.24133819e+01  1.33671108e+01